Testing Propotype with Selenium & google library
*** The Following must using selenium with FireFox web driver. 
*** Be sure that Firefox has been installed on the local machine.
*** The Script may only running on LOCAL MACHINE ONLY

In [ ]:
# Install Library
pip install selenium pandas google


In [ ]:
from googlesearch import search
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from urllib.parse import urlparse
import pandas as pd
import time
import traceback
import re

OUTPUT_FILE = 'output.csv'
AUTHOR_MAX_WORDS = 3
SLEEP_INTERVAL = 2

p_elementList= []

# Cuntion for Getting Date from Content
def extractDate(string):
    try:
       # Define a regular expression for the date format DD/MM/YYYY and optional time
        date_time_pattern = re.compile(
            r"^\b\d{1,2}[-/]\d{1,2}[-/]\d{2,4}\b"  # Matches dates like DD/MM/YYYY at the start of the string
            r"|^\b(?:January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{1,2},\s+\d{4}\b"  # Matches dates like December 20, 2023
            r"|^\b(?:January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{1,2},\d{4}\b"  # Matches dates like December 20, 2023
            r"|^\b(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s+\d{1,2},\s+\d{4}\b"  # Matches dates like Dec 20, 2023
            r"|^\b(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s+\d{1,2},\d{4}\b"  # Matches dates like Dec 20, 2023
            r"(?:\s+\b\d{1,2}:\d{2}(?::\d{2})?\s*(?:AM|PM)?\s*(?:ET|WT)?\b)?$",  # Optionally matches times with optional AM/PM and time zone
            re.IGNORECASE  # Make the pattern case-insensitive
        )
        # Check if the entire string matches the pattern
        match = date_time_pattern.match(string)
        return bool(match)
    except:
        print("Error accessing a text:", e)
    return False

# Function for Getting Content from url
def getContent(driver,url):
    try:
        parsed_url = urlparse(url)

        authors = '' 
        source_domain = parsed_url.netloc
        copyright = ''
        paragraph = ''
        title = ''
        date = ''
        driver.get(url)
        H1_elements = driver.find_elements(By.TAG_NAME, "h1")
        p_elements = driver.find_elements(By.TAG_NAME, "p")
            
        # Getting Title
        for h1 in H1_elements:
            try:
                title = h1.text
                break  # Assuming you only need the first h1's text
            except Exception as e:
                print("Error accessing h1 text:", e)
                continue

        # Getting Content
        for p in p_elements:
            try:
                value = p.text
                # Filter out authors
                if ((value.lower().strip().startswith('written by:')) or\
                    (value.lower().strip().startswith('article by:'))
                    ):
                    value.replace('Article by:', '')
                    authors = value.replace('Written by:', '') 
                    #authors = value.lower().replace('by ', '') 
                    #words = authors.split()
                    #if len(words) <= AUTHOR_MAX_WORDS and not authors:
                    #    authors = value
                    #else:
                    #    authors = ''
                    continue
                # Filter out Copyright
                if '©' in value or 'All Rights Reserved' in value:
                    copyright = value
                    continue

                # call date function to extract the date
                if ((value.lower().strip().startswith('updated on')) or \
                    (value.lower().strip().startswith('published on')) or \
                    (value.lower().strip().startswith('published online')) 
                     ):
                    value = value.replace('Updated on', '').replace(':', '') 
                    value = value.replace('Published on', '').replace(':', '') 
                    value = value.replace('Published Online', '').replace(':', '') 
                    if (extractDate(value.strip())):
                        date = value
                        continue

                paragraph += value + ' '
            except Exception as e:
                print(f"Error accessing paragraph text from {url}:\n", e)
                continue
    except TimeoutException:
        authors = ''
        print(f"Timeout waiting for elements on the page: {url}")
    except NoSuchElementException:
        authors = ''
        print(f"An element was not found on the page: {url}")
    except Exception as e:
        authors = ''
        print(f"An unexpected error occurred with URL => {url}: {e}")

    return {'authors': authors, 'date' : date, 'title' : title, 'content': paragraph, 'source_domain': source_domain, 'url' : url, 'copyright': copyright } 

def getPElementList(driver,url):
    try:
        parsed_url = urlparse(url)

        authors = '' 
        source_domain = parsed_url.netloc
        copyright = ''
        paragraph = ''
        title = ''
        readMoreUrl = ''
        date = ''
        driver.get(url)
        p_elements = driver.find_elements(By.TAG_NAME, "p")

        # Getting Content
        for p in p_elements:
            try:
                value = p.text
                p_elementList.append({'p_element': value,'title' : title,  'source_domain': source_domain, 'url' : url, })

            except Exception as e:
                print(f"Error accessing paragraph text from {url}:\n", e)
                continue
    except TimeoutException:
        authors = ''
        print(f"Timeout waiting for elements on the page: {url}")
    except NoSuchElementException:
        authors = ''
        print(f"An element was not found on the page: {url}")
    except Exception as e:
        authors = ''
        print(f"An unexpected error occurred with URL => {url}: {e}")

    return 

def save_to_csv_with_pandas(recordSet, filename):
    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(recordSet)
    # Save the DataFrame to a CSV file
    df.to_csv(filename, index=False, encoding='utf-8')


Please run and input your query

In [ ]:
print("Which query content do you want to search?")
queryString = input()

#Testing query
#queryString = 'Did Joe Biden tell voters to stay home and not vote?'
#queryString = 'Is Michael Saylor saying he will give away free bitcoin?'
#queryString = 'Is Taylor Swift going to give away a free Le Creuset cookware set?'
#queryString = 'Did Elon Musk say you can control a computer mouse with neuralink?'
#queryString = 'Oprah winfrey weight loss gummies?'
#queryString = 'Did Justin Trudeau say invest in Petro canada?'
#queryString = 'How to enter Jennifer Aniston MacBook giveaway?'

Run following command to get result from google search

In [ ]:
#Search in google url with google library
results = search(query=queryString,num = 10 )

Run following script to get DataFrame result

In [ ]:
dataSet = []
driver = webdriver.Firefox()
i = 0
for url in results:
    print(f'Scripting from => {url}')
    try:
        data = getContent(driver,url)
        print(data)
        if (data['title'] !='' or data['content'] != ''):
            content = data['content']
            if (content.strip().startswith('Log In')):
                print("content Excluded!!")
            else:
                dataSet.append(data)
    except:
        print(f"An unexpected error occurred with URL => {url}: {e}")
    #time.sleep(SLEEP_INTERVAL)
driver.close()
df = pd.DataFrame(dataSet)


export Dataframe to CSV if you need

In [19]:
save_to_csv_with_pandas(df, queryString.replace('?','.')+ 'csv')